In [ ]:
import math
import numpy as np
import cupy as cp
# import tensorflow as tf
import cv2
import scipy.sparse as ssp
from scipy import ndimage
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display
import package.myUtil as myUtil

In [ ]:
def interpolate_frames_lk(frames):
    """
    Lucas-Kanade法（疎なオプティカルフロー）を用いてフレーム補間を行い、
    オプティカルフローを可視化します。

    :param frames: numpy.ndarray, 形状は (m, 128, 128)
    :return: tuple
        - numpy.ndarray, 形状は (m-1, 128, 128) の補間フレーム
        - list of numpy.ndarray, オプティカルフローを可視化した各フレーム（カラー画像）
    """
    m, h, w = frames.shape
    new_frames = []
    flow_visualizations = []

    # 輝度値のスケーリング係数を計算
    # 全フレームの最大値を用いて0〜255にスケーリング
    max_value = np.max(frames)
    min_value = np.min(frames)
    print("max_value: ", max_value, "min_value: ", min_value)
    scale_factor = 255.0 / max_value
    max_corners = 10

    for i in range(m - 1):
        # 特徴点の検出
        frame1 = frames[i]
        frame2 = frames[i + 1]
        frame1[frame1 < 1e-4] = 0
        frame2[frame2 < 1e-4] = 0
        frame1 = (frame1 * scale_factor).astype(np.uint8)
        frame2 = (frame2 * scale_factor).astype(np.uint8)

        prev_points = cv2.goodFeaturesToTrack(
            image=frame1,
            maxCorners=max_corners,
            qualityLevel=0.05,
            minDistance=5,
            mask=None,
            blockSize=5,
            useHarrisDetector=False,
        )

        if prev_points is None:
            interpolated_frame = (frames[i] + frames[i + 1]) / 2.0
            new_frames.append(interpolated_frame)
            flow_visualizations.append(
                cv2.cvtColor((interpolated_frame * scale_factor).astype(np.uint8), cv2.COLOR_GRAY2BGR)
            )
            continue

        # 特徴点の追跡
        next_points, status, error = cv2.calcOpticalFlowPyrLK(
            prevImg=frame1,
            nextImg=frame2,
            prevPts=prev_points,
            nextPts=None,
            winSize=(15, 15),
            maxLevel=2,
            criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03),
        )

        # 有効なポイントのみを選択
        status = status.reshape(-1)
        valid_idx = status == 1
        prev_pts_valid = prev_points.reshape(-1, 2)[valid_idx]
        next_pts_valid = next_points.reshape(-1, 2)[valid_idx]

        # print points
        for idx, ((x0, y0), (x1, y1)) in enumerate(zip(prev_pts_valid, next_pts_valid)):
            print("point[", idx, "]: ", x0, y0, " -> ", x1, y1)

        # オプティカルフローの平均を計算
        flow_vectors = next_pts_valid - prev_pts_valid
        mean_flow_vector = np.mean(flow_vectors, axis=0)
        print("mean_flow_vector: ", mean_flow_vector)

        # 平均ベクトルの半分だけframe1を移動
        interpolated_frame = np.zeros_like(frames[i])
        interpolated_frame = np.roll(frames[i], int(round(mean_flow_vector[0] / 2)), axis=1)
        interpolated_frame = np.roll(interpolated_frame, int(round(mean_flow_vector[1] / 2)), axis=0)

        new_frames.append(interpolated_frame)

        # オプティカルフローの可視化
        flow_vis = cv2.cvtColor(frame1, cv2.COLOR_GRAY2BGR)

        # ランダムな色を生成
        np.random.seed(42)  # 再現性のためシードを固定
        colors = np.random.randint(0, 255, (max_corners, 3))

        for idx, ((x0, y0), (x1, y1)) in enumerate(zip(prev_pts_valid, next_pts_valid)):
            color = tuple(map(int, colors[idx % max_corners]))
            start_point = (int(round(x0)), int(round(y0)))
            end_point = (int(round(x1)), int(round(y1)))
            cv2.arrowedLine(flow_vis, start_point, end_point, color, 1, tipLength=0.3)

        flow_visualizations.append(flow_vis)

    return np.stack(new_frames), flow_visualizations

In [ ]:
def matrix_to_tensor(H, m, n):
    return H.reshape(m, m, n, n)


def tensor_to_matrix(tensor, m, n):
    return tensor.reshape(m * m, n * n)


n = 128
_m = 128
DATA_PATH = "../../OneDrive - m.titech.ac.jp/Lab/data"
# DATA_PATH = "../data"
CAP_DATE = "241114"
EXP_DATE = "241118"
H_SETTING = f"p-5_lmd-100_m-{_m}"
H_mat = np.load(f"{DATA_PATH}/{EXP_DATE}/systemMatrix/H_matrix_{H_SETTING}.npy")
print("H shape:", H_mat.shape, "type(H):", type(H_mat), "H.dtype:", H_mat.dtype)
H_ten = matrix_to_tensor(H_mat, _m, n)
H_ten[:, :, 0, 0]=0

In [ ]:
# フレーム補間テスト
frames = np.array([H_ten[64, 32, :, :], H_ten[64, 42, :, :]])
interpolated_frames, flow_visualizations = interpolate_frames_lk(frames)
# plt.imshow(frames[0], cmap="gray")
# plt.colorbar()
# オプティカルフローを表示
for idx, flow_vis in enumerate(flow_visualizations):
    plt.figure(figsize=(4, 4))
    plt.imshow(cv2.cvtColor(flow_vis, cv2.COLOR_BGR2RGB))
    plt.show()

In [ ]:
print("interpolated_frames.shape: ", interpolated_frames.shape)
plt.figure(figsize=(4, 4))
plt.imshow(interpolated_frames[0], cmap="gray")
# plt.imshow(frames[1], cmap="gray")

In [ ]:
# フレーム補間
m = 2 * _m - 1  # 255
interpolated_tensor = np.zeros((m, m, n, n), dtype=np.float32)
for i in range(0, m, 2):
    frames = H_ten[i // 2, :, :, :]
    interpolated_frames = interpolate_frames_lk(frames)
    for j in range(m):
        if j % 2 == 0:
            interpolated_tensor[i, j, :, :] = H_ten[i // 2, j // 2, :, :]
        else:
            interpolated_tensor[i, j, :, :] = interpolated_frames[j // 2, :, :]
for j in range(0, m):
    frames = interpolated_tensor[::2, j, :, :]
    interpolated_frames = interpolate_frames_lk(frames)
    for i in range(1, m, 2):
        interpolated_tensor[i, j, :, :] = interpolated_frames[i // 2, :, :]

interpolated_matrix = tensor_to_matrix(interpolated_tensor, m, n)

In [ ]:
# np.save(f"{DATA_PATH}/{EXP_DATE}/systemMatrix/H_matrix_int_{H_SETTING}.npy", interpolated_matrix)
print("H shape:", interpolated_matrix.shape, "type(H):", "H.dtype:", interpolated_matrix.dtype)
# print(f"Saved {DATA_PATH}/{EXP_DATE}/systemMatrix/H_matrix_int_{H_SETTING}.npy")

In [ ]:
SAMPLE_NAME = "Cameraman"
sample_image = Image.open(f"{DATA_PATH}/sample_image{n}/{SAMPLE_NAME}.png").convert("L")
sample_image = cp.asarray(sample_image).ravel() / 255

black = myUtil.calculate_bias(m**2, DATA_PATH, CAP_DATE)

H = cp.asarray(interpolated_matrix)
Hf = H @ sample_image + black
Hf = cp.asnumpy(Hf.reshape(m, m))
print("Hf shape:", Hf.shape)

Hf_pil = Image.fromarray((Hf * 255).astype(np.uint8), mode="L")
Hf_pil.save(f"{DATA_PATH}/{EXP_DATE}/Hf_int_lk.png", format='PNG')
display(Hf_pil)

plt.imshow(Hf, cmap='gray', interpolation='nearest')
plt.colorbar()
plt.title('Grayscale Heatmap')
plt.show()

In [ ]:
I = 64
# J = 128
interpolated_frames = interpolated_tensor[I, :, :, :]
print("interpolated_frames.shape:", interpolated_frames.shape)
# 表示したいフレームの範囲を設定
start_index = 60
end_index = 84
frames_to_display = interpolated_frames[start_index:end_index + 1]

num_frames = len(frames_to_display)
cols = 5  # 列数を設定（必要に応じて変更可能）
rows = math.ceil(num_frames / cols)  # 行数を自動計算

# サブプロットを作成
fig, axs = plt.subplots(rows, cols, figsize=(5 * cols, 5 * rows))

# サブプロットの軸を1次元にフラット化してループ処理
for idx, ax in enumerate(axs.flat):
    if idx < num_frames:
        frame = frames_to_display[idx]
        img = ax.imshow(frame, cmap='gray')
        plt.colorbar(img, ax=ax)
        ax.set_title(f'Frame {start_index + idx}')
    else:
        ax.axis('off')  # 使用しないサブプロットを非表示に

In [ ]:
import imageio
import io

def create_heatmap_gif(data, output_path, duration=0.5):
    """
    m×n×nのnumpy配列をヒートマップのGIFに変換する。

    Parameters:
    - data: ndarray, shape (m, n, n)
    - output_path: str, 保存するGIFのパス
    - duration: float, 各フレームの表示時間（秒）
    """
    # データが3次元か確認
    if data.ndim != 3:
        raise ValueError("入力データは3次元のnumpy配列である必要があります。")

    m, n1, n2 = data.shape
    if n1 != n2:
        raise ValueError("各スライスはn×nの形状である必要があります。")

    # テンソルの最大値と最小値を計算
    vmin = np.min(data)
    vmax = np.max(data)

    images = []  # GIFに追加する画像のリスト

    # 図と軸を設定
    fig, ax = plt.subplots(figsize=(6, 6))
    # plt.axis('off')  # 軸を非表示

    # ヒートマップを初期化
    cax = ax.imshow(data[0], cmap='gray', vmin=vmin, vmax=vmax)
    # カラーバーを一度だけ追加
    cbar = fig.colorbar(cax, ax=ax, fraction=0.046, pad=0.04)
    cbar.ax.tick_params(labelsize=8)
    title = ax.set_title(f'Frame 1/{m}', fontsize=10)

    for i in range(m):
        # ヒートマップのデータを更新
        cax.set_data(data[i])
        # タイトルを更新
        title.set_text(f'Frame {i+1}/{m}')

        # レイアウトを固定
        fig.tight_layout()

        # 画像をバッファに保存
        buf = io.BytesIO()
        plt.savefig(buf, format='png')
        buf.seek(0)
        image = imageio.v2.imread(buf)
        images.append(image)
        buf.close()

    plt.close(fig)  # 図を閉じる

    # GIFとして保存
    imageio.mimsave(output_path, images, duration=duration)
    print(f"GIFが保存されました: {output_path}")

create_heatmap_gif(interpolated_frames, f"{DATA_PATH}/{EXP_DATE}/interpolated_i{I}_video.gif", duration=0.2)